# Exercise 1: Algorithmic and Search Complexity

_course: the real blockchain
<br>date: 2 september 2020
<br>update: 3 september 2020
<br>author: burton rosenberg_


## Overview


Security is a matter of chance. In most instances of securing a device, whether it be a password, a physical
key, or even a safe, since there is some valid way to open the secured device, a determined attacker can open the secured device. This can either be by,


- a methodical search, of one key after the other,
- randomly guessing the key, 
- calculating the key given a weakness in the system.

It is important if the system is secure that the first two options be slow in a very meaningful way, and that the thir way does not exist.

Facing a methodical search, we want that the serach be exponentially lengthy in the size of the key. There are 26^10 different passwords on ten letters. The good guy only needs to recite ten letters, the bad guy needs to search (on average) 26^10/2 possibilities.

Facing a random guess, we want that the probability of guessing correctly is exponentially small in the size of the key. In the case of our 10 letter password, the attacker's probability of guessing the key (in one try, assuming a "good" key) is miniscule: 1/26^10.

Facing a weakness in the system, we will suppose that a computer algorithm exploits the weakness (so we are not talking about _social engineering_ or other attacks, just "laboratory conditions") is sufficently inefficient as to not be an improvement on the first attack &mdash; methodical search.


## Algorithm Complexity

We explore first the theory supporting the last case.

We have to measure the run time of the algorithm, similar to how we measure methodical search by counting something and making a broad statement that that count is a sufficient indication of run time. In methodical 
search we count key tries, and assume each key try takes a fixed amount of time. More might need be done 
than try the key, but we are content with just accounting for key tries.

In an algorithm, we look also for some countable action that we can use as a measure of run time. In this
exercise, we will sort numbers, and we will count the number of comparisons between two numbers. It is
pretty certain that to sort we need to compare numbers, and we feel justified in thinking the comparison
must take some fixed amount of time, and we are happy letting any other computation be given for free.


In [1]:
import numpy as np
import math
import random



class QueryCompare:
    
    def __init__(self):
        self.count = 0
        
    def reset_count(self):
        self.count = 0
        
    def compare_gt(self,a,b):
        self.count += 1
        return a > b

    def compare_le(self,a,b):
        self.count += 1
        return a <= b



### Exercise A: Selection Sort. 

Selection sort is a well known "slow" sort. 

Given an unsorted list of numbers, the algorithm considers the 
prefix of that list, beginning with the prefix of one element, then two elements, and so on. When it is 
done considering a prefix, those numbers in the prefix have been placed in order. Therefore when it begins
considering a prefix, all numbers but the last are assurdedly in order, and the last might be in order too.
Else the last number is moved downwards among the numbers in the list until it finds its place.

Just counting comparisons, give a very good formula the predicts ths number of comparison queries made to sort the list, as a function of list size.

__To do:__ Experiment to get various runs values for different list lengths $n$. The number of queries is not a strict function of $n$ &mdash; it can vary depending on how the input is arranged. That is a property of the selection sort algorithm. The exact number of queries is data dependent. Then fit a curve to the data. 

Since we know the result is a quadratic, the curve to fit has the form,
$$
f(n) = a \, n^2+b \,n+c
$$
optimizing over $a, b$ and $c$.

The numpy function numpy.polynomial.polynomial.Polynomial.fit can help.



In [2]:

class SelectionSort:
    
    def __init__(self, query_compare_object):
        self.qc = query_compare_object

    def swap(self,a,i,j):
        t = a[i]
        a[i] = a[j]
        a[j] = t

    def sort(self,x):
        n = len(x)
        self.qc.reset_count()
        for i in range(n):
            for j in range(i,0,-1):
                if self.qc.compare_le(x[j],x[j-1]):
                    self.swap(x,j,j-1)
        return x
    
    def test_sort(self,n=16):
        test_array = [random.randint(0,n*n) for i in range(n)]
        self.sort(test_array)
        return (self.qc.count,test_array)

ss = SelectionSort(QueryCompare())
test = ss.test_sort()
print(f'length:\t{len(test[1])},\nquery:\t{test[0]},\nsorted list:\n{test[1]})')



length:	16,
query:	120,
sorted list:
[9, 35, 58, 77, 91, 105, 109, 114, 155, 157, 197, 221, 228, 231, 232, 237])



### Exercise B: Merge Sort. 

Merge sort is a well known "fast" sort. 

Given an unsorted list of numbers, the algorithm breaks the list in two in the middle. It sorts the first
half recursively, sorts the second half recursively, then merges back together the two halves. The merge
consumes the two sorted half lists to build the sorted combined list by looking at the elements at the top
of the halves, removes which is smaller and places that element onto the end of the built list. 

There will be cases in which all of one half has been consumed yet more remain on the other list, and of course
then no choice is made, the element is simplied transferred from the ever dwindleing half list to the ever growing combined list.

Just counting comparisons, give a very good formula the predicts ths number of comparison queries made to sort the list, as a function of list size.

__To do:__ Experiment to get various runs values for different list lengths $n$. The number of queries will be a strict function of $n$ &mdash; for merge sort, the exact number of queries is _not_ data dependent. Then fit a curve to the data. 


Since we know the function is $O(n\,\log(n))$, the curve to fit has the form,
$$
f(n) = a \, \log n+b \,n+c
$$
optimizing over $a, b$ and $c$.


The scipy function scipy.optimize.curve_fit can help.


In [3]:
   
class MergeSort:
    
    def __init__(self, query_compare_object):
        self.qc = query_compare_object
    
    def merge(self,x,y):
        z = [0]*(len(x)+len(y))
        i, j = 0, 0
        for k in range(len(z)):
            if i>=len(x):
                z[k] = y[j]
                j += 1
                continue
            if j>=len(y):
                z[k] = x[i]
                i += 1
                continue
            if self.qc.compare_le(x[i],y[j]):
                z[k] = x[i]
                i += 1
            else:
                z[k] = y[j]
                j += 1
        return z
            
    def sort_aux(self,x):
        n = len(x)
        if n<2:
            return x
        x_bot = self.sort_aux(x[:n//2])
        x_top = self.sort_aux(x[n//2:])
        return self.merge(x_bot,x_top)
        
    def sort(self,x):
        self.qc.reset_count()
        return self.sort_aux(x)
    
    def test_sort(self,n=16):
        test_array = self.sort([random.randint(0,n*n) for i in range(n)])
        return (self.qc.count,test_array)

ms = MergeSort(QueryCompare())
test = ms.test_sort()
print(f'length:\t{len(test[1])},\nquery:\t{test[0]},\nsorted list:\n{test[1]})')

 
    

length:	16,
query:	43,
sorted list:
[7, 30, 34, 39, 41, 58, 95, 96, 156, 159, 166, 167, 179, 198, 201, 214])


## Search and Entropy

We are now going to think about how to guess a key, and how to quantify the effort that is expected to guess a key. In the greater context, the security of a protocol is a quote of this difficulty. Since there exists a password, a combination to a safe, or a cut key that opens the lock, it is not possible to say that password system, safe, or lock, is unopenable. Something opens it, and the attacker can discover, one way or the other, the details of what opens it. What we need is to understand and quantify is the difficulties that lie in the way of the attacker's discovery.

In code, we will model the key trials as an object QuestionBox which contains a secret, and which can be queried for a True/False answer, about the key. An equality query answers True if the presented key is the secret, and False otherwise. An inequality quere, such as $\le$, answers True if the presented key is $\le$ the secret, and False otherwise.

For this second sort of query to make sense, we must assume that the keys have an ordering, and less-than is interpreted as a key coming before another in that ordering. They keys might be identified with an integer index, and one key is less than another if its index is less than the other key's index.

We will also need probability distributions. These will be represented as objects, that return the probabiity of a key given the key. Or if preferred, rather that the key itself that is presented, it is the index of the key. In this way we can think of keys as being the set of numbers $\{\,0,1,\ldots,n-1\,\}$ when in reality they might be various more interesting things &mdash; cuts on a metal bar, combinations for a combination lock, and so on.

We look at two probability distributions on elements. Both distributions are on a key set represented by integers 0 through $n-1$. We organize the elements in descending probability. If $p_i$ is the probability of item $i$, the $p_i \ge p_j$ whenever $i\le j$.

The _truncated geometric distribution_ takes a probability $p_i$ of element $i$ the probability of tossing $i$ heads before the first tail, with a coin of bias $p$. 

The _single favorite distribution_ sets $p_0 = p$ and the remaining elements 1 through $n-1$ have probabilities each of $(1-p)/(n-1)$. 



In [4]:
import numpy as np
import math
import random

        
class TruncGeometric:
    
    # implements a PMF with monotonicaly descending probability values
    # support in 0 to n-1
    
    def __init__(self,n,p):
        self.n = n
        self.p = p
        self.c = p*(1-math.pow(p,n))/(1-p)

    def p_i(self,i):
        return math.pow(self.p,i+1)/self.c


class SingleFavorite:
        
    # implements a PMF with monotonicaly descending probability values
    # support in 0 to n-1

    def __init__(self,n,p):
        self.n = n
        self.p = p
        self.c = (1.0-p)/(n-1)
    
    def p_i(self,i):
        if i==0:
            return self.p
        return self.c

class QuestionBox:
    
    def __init__(self,pmf):
        self.pmf = pmf
        self.secret = 0

    def check_normalized(self,pmf):
        s = 0
        for i in range(pmf.n):
            s += pmf.p_i(i)
        assert(math.isclose(s,1.0))
        
    def choose_a_number(self):
        r = random.random()
        #  between 0 and 1
        cumdist = 0.0
        for i in range(0,self.pmf.n):
            cumdist += self.pmf.p_i(i)
            if cumdist >= r:
                self.secret = i
                return i
        return self.pmf.n-1
        
    def query_equal(self,guess):
        return guess == self.secret

    def query_le(self,guess):
        return guess >= self.secret
    




### Exercise C: Most Likely First Sequential

A typical attack againts a key is to sequentially try all keys until the correct key is found. When a key is chosen, often the choice is made with a bias. Passwords are notorious for being chosen with bias. They are often a birthday or some clever "secret" word, like AstroStarDog. We will assume that the attacker has exact knowledge of this bias. Therefore, the impatient attacker should try the likely keys first, as this will speed key discovery.

If the keys are $\{\,0,1,\ldots,n-1\,\}$, and the probability of key $i$ being chosen is $p_i$, and we arrange the keys so that $p_i \ge p_j$ when $i<j$, then the attacker acheives expected time,
$$
\sum_{i=0}^{n-1} p_i \, (i+1)
$$
by asking a equality query on the keys in the order  $,0,1,\ldots,n-1$. This formula is called the _guessing entropy_.

- Complete the code for the guessing entropy function. 

- Implement the most likely first sequential search and test on the two distributions.

- Compare the average number of queries for most likely first search against the guessing entropy.

### Exercse D: Binary Search

It might be possible that inequality queries are permitted against a key. Such a query can eleminate with a single query many candidates. Although it is not typical that an attacker is permitted such a powerful query, it does happen. (See story below.)

In this case the optimal approach is to choose a query that divides the remaining candidates into two equaly halves. With a single query one of the two halves are eliminated. Continuing in this way, in about $\log n$ queries the key is discovered. 

- Complete the code for the hartley entropy function ($\log n$).

- Implement the binary search strategy and test on the two distributions.

- Compare the average number of queries for binary search against the hartley entropy.

### Exercise E: Balanced Search

Binary search does not make use of any bias in the key selection. Rather than dividing keys into two equal sets, where equality is by number of keys, try dividing them into two equal sets, where equality is by the sum of the probability of the keys in each part. 

As the keys are indexed by decreasing likelyhood, this means, for example, that the first query is at location $k$ where $k$ is chosen such that,
$$
1/2 \approx \sum_{i=0}^k p_i
$$
Subsequent queries are calculated in the same spirit. 

The result should be likely keys are discovered with fewer queries than unlikely keys. Overall the average number of queries will be the weighted sum, weighted by probability, of what looks like the simple Hartley entropy of log the number of keys at that probability level. That is, 
$$
H(D) = - \sum_{i=0}^{n-1} p_i \log p_i
$$
This is the _Shannon Entropy_. However it is really "the" entropy, so it is just called Entropy. With the logarithm in the base 2, the result has dimension of bits.

- Complete the code for the shannon entropy function.

- Implement the balanced search strategy and test on the two distributions.

- Compare the average number of queries for binary search against the shannon entropy.

In [4]:

class SearchStrategy:
    
    def __init__(self,question_box):
        self.qb = question_box
        self.pmf = question_box.pmf
        self.n = question_box.pmf.n

    def make_cdf(self,pmf):
        cdf = [pmf.p_i(i) for i in range(pmf.n)]
        for i in range(1,pmf.n):
            cdf[i] += cdf[i-1]
        cdf[pmf.n-1] = 1.0 # avoid roundoff problems
        return cdf

    def shannon_entropy(self):
        se = 0
        
        # code here
        
        return -se

    def guessing_entropy(self):
        ge = 0
        
        # code here 

        return ge

    def hartley_entropy(self):
        support = 0
        
        # code here
        
        return math.log2(support)

    def most_likely_first(self):
        # tries values using most likely first strategy and
        # question box equality queries until the secret is
        # discovered.
        
        # code here
        
        return False

    def most_likely_first_avg(self,trials=1000):
        tot_queries = 0
        for i in range(trials):
            magic = self.qb.choose_a_number()
            (m,t) = self.domain_sweep()
            tot_queries += t
            assert (m==magic)
        return tot_queries/trials
    
    def binary_search(self):
        # tries values using binary search and question
        # box inequality queries until the secret is 
        # discovered
        
        # code here 
        
        return False
    
    def binary_search_avg(self,trials=1000):
        tot_queries = 0
        for i in range(trials):
            magic = self.qb.choose_a_number()
            (m,t) = self.binary_search()
            tot_queries += t
            assert (magic==m)
        return tot_queries/trials

    def balanced_search(self):
        # tries values using balanced search and question
        # box inequality queries until the secret is 
        # discovered
        
        # code here
        
        return False
            
    def balanced_search_avg(self,trials=1000):
        tot_queries = 0
        for i in range(trials):
            magic_number = self.qb.choose_a_number()
            #print(f'magic number: {magic_number}')
            (mn,nq) = self.balanced_search()
            assert(mn==magic_number)
            tot_queries += nq
        return tot_queries/trials            
 

In [6]:

n = 64
p = 0.7
print(f'\ntruncated geometric distribution. n={n} and prob={p}')
ss = SearchStrategy(QuestionBox(TruncGeometric(n,p)))
print(f'\tlog n: {math.log(n):.3f},\tbinary search queries: {ss.binary_search_avg()}')
print(f'\tguessing entropy: {ss.guessing_entropy():.3f},\tmost likely first queries: {ss.domain_sweep_avg():.3f}')
print(f'\tshannon entropy: {ss.shannon_entropy():.3f},\tbalanced search queries: {ss.balanced_search_avg():.3f}')


n = 64
p = 0.7
print(f'\nsingle favorite distribution. n={n} and prob={p}')
ss = SearchStrategy(QuestionBox(SingleFavorite(n,p)))
print(f'\tlog n: {math.log(n):.3f},\tbinary search queries: {ss.binary_search_avg()}')
print(f'\tguessing entropy: {ss.guessing_entropy():.3f},\tmost likely first queries: {ss.domain_sweep_avg():.3f}')
print(f'\tshannon entropy: {ss.shannon_entropy():.3f},\tbalanced search queries: {ss.balanced_search_avg():.3f}')

#print(f'cumulative distribution function: {ss.pmf.get_cdf()}')



truncated geometric distribution. n=64 and prob=0.7
	log n: 4.159,	binary search queries: 5.915
	guessing entropy: 3.333,	most likely first queries: 3.380
	shannon entropy: 2.938,	balanced search queries: 3.261

single favorite distribution. n=64 and prob=0.7
	log n: 4.159,	binary search queries: 6.476
	guessing entropy: 10.600,	most likely first queries: 11.046
	shannon entropy: 2.674,	balanced search queries: 2.748


<pre>
sample output of the exercises

truncated geometric distribution. n=64 and prob=0.7
	log n: 4.159,	binary search queries: 5.915
	guessing entropy: 3.333,	most likely first queries: 3.380
	shannon entropy: 2.938,	balanced search queries: 3.261

single favorite distribution. n=64 and prob=0.7
	log n: 4.159,	binary search queries: 6.476
	guessing entropy: 10.600,	most likely first queries: 11.046
	shannon entropy: 2.674,	balanced search queries: 2.748
</pre>

### Epilogue: A True Story

How useful is the knowledge presented in this exercise? A true story ...

At some private university in South Florida that had a directory service allowing data lookup. Although you could do a lookup on social security number, if you knew it, only the last 4 digits were ever presented, so the remaining 5 digits were considered securely secret. 

The designers apparantly were thinking along the lines of Guessing Entropy and equality queries. This would give an entropy as high as $10^5/2$, about a quarter of a million queries.

However, it was overlooked was that "prefix search" on social security numbers was permited. While not exactly an inequality query, the differences are unimportant in effect. 

How would you use the principles of binary search and inequality queries to lower the expected number of queries to discover a particular social security number from the believed quarter of a million queries, to only 25 queries?

The flaw was reported and patched.
